In [2]:
%load_ext autoreload
%autoreload 2
from ripser import ripser
from persim import plot_diagrams
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import cv2                  
import numpy as np  
from tqdm import tqdm
import os                   
from random import shuffle  
from zipfile import ZipFile
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
#preprocess.
from keras.preprocessing.image import ImageDataGenerator

#dl libraraies
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical

# specifically for cnn
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Dropout 
import tensorflow as tf
import random as rn

Using TensorFlow backend.


In [8]:
from skimage import data
from skimage.color import rgb2gray
IMG_SIZE=50
def make_train_data(DIR):
    for img in os.listdir(DIR):
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        img = rgb2gray(img)
        ret, b_img = cv2.threshold(img,.4,1,cv2.THRESH_BINARY)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        t=np.array(b_img)
        t1=[]
        for i in range(50):
            for j in range(50):
                if(t[i][j]==1):
                    t1.append([i,j])
        X.append(t1)

In [9]:
X=[]
DIR='/Users/user/Desktop/Covid-19-Pnumonia-dataset'

In [10]:
make_train_data(DIR)
print(len(X))

79


In [11]:
from skimage import color
from skimage import io
import numpy as np
from ripser import ripser
from persim import plot_diagrams
import gudhi

In [12]:
l5=[]
l6=[]
def barcodes1(diag):
        for j in range(len(diag)):
            birth=diag[j][1][0]
            deth=diag[j][1][1]
            if((deth-birth)>.1):
              if(diag[j][0]==0):
                l5.append([birth,deth])
              if(diag[j][0]==1):
                l6.append([birth,deth])

In [13]:
l3=[]
l4=[]
def barcodes(diag):
        for j in range(len(diag)):
            birth=diag[j][1][0]
            deth=diag[j][1][1]
            if((deth-birth)>.1):
              if(diag[j][0]==0):
                l3.append([birth,deth])
              if(diag[j][0]==1):
                l4.append([birth,deth])

In [14]:
def betti_nums1(data, eps):
    diagrams = ripser(data, maxdim = 1,distance_matrix=False, metric = 'euclidean',thresh =5)['dgms']
    b=[]
    c=0
    for j in range(len(diagrams[0])):
        if(diagrams[0][j][1]-diagrams[0][j][0]>.1):
            c+=1
    b0=c
    c=0
    for j in range(len(diagrams[1])):
        if(diagrams[1][j][1]-diagrams[1][j][0]>.01):
            c+=1
    b1=c   
    b.append(b0)
    b.append(b1)
    #plot_diagrams(diagrams)  
    #barcodes(diagrams)
    return b

In [15]:
import numpy as np
import math
import random
import gudhi
from sklearn.neighbors import KDTree
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pylab as pl
Y=0
for i in range(len(l1[0])):
       x=[l1[0][i],l1[1][i]]
       y=[Y,Y]
       Y=Y+5
       if(i==len(l1[0])-1):
         plt.plot(x, y,c='b',label='H0')
       else:
         plt.plot(x, y,c='b')
for i in range(len(l2[0])):
       x=[l2[0][i],l2[1][i]]
       y=[Y,Y]
       Y=Y+5
       if(i==len(l2[0])-1):
         plt.plot(x, y,c='r',label='H1')
       else:
         plt.plot(x, y,c='r')
plt.title("Homology Barcodes")
plt.xlabel('time')
pl.legend()
pl.show()

In [16]:
X1=[]
for i in range(len(X)):
         X[i]=np.array(X[i])
         X1.append(betti_nums1(X[i], 1.42 ))

In [17]:
def DTM(X,query_pts,m):
    N_tot = X.shape[0]     
    k = math.floor(m*N_tot)+1   # number of neighbors
    kdt = KDTree(X, leaf_size=30, metric='euclidean')
    NN_Dist, NN = kdt.query(query_pts, k, return_distance=True)  
    DTM_result = np.sqrt(np.sum(NN_Dist*NN_Dist,axis=1) / k)
    return(DTM_result)
def Filtration_value(p, fx, fy, d, n = 10):
    if p==np.inf:
        value = max([fx,fy,d/2])
    else:
        fmax = max([fx,fy])
        if d < (abs(fx**p-fy**p))**(1/p):
            value = fmax
        elif p==1:
            value = (fx+fy+d)/2
        elif p==2:
            value = np.sqrt( ( (fx+fy)**2 +d**2 )*( (fx-fy)**2 +d**2 ) )/(2*d)            
        else:
            Imin = fmax; Imax = (d**p+fmax**p)**(1/p)
            for i in range(n):
                I = (Imin+Imax)/2
                g = (I**p-fx**p)**(1/p)+(I**p-fy**p)**(1/p)
                if g<d:
                    Imin=I
                else:
                    Imax=I
            value = I
    return value
def WeightedRipsFiltration(X, F, p, dimension_max , filtration_max):
    N_tot = X.shape[0]     
    distances = euclidean_distances(X)         
    st = gudhi.SimplexTree()                    

    for i in range(N_tot):                  
        value = F[i]
        if value<filtration_max:
            st.insert([i], filtration = F[i])            
    for i in range(N_tot):                  
        for j in range(i):
            value = Filtration_value(p, F[i], F[j], distances[i][j])
            if value<filtration_max:
                st.insert([i,j], filtration  = value)
    
    st.expansion(dimension_max)                
 
    result_str = 'Weighted Rips Complex is of dimension ' + repr(st.dimension()) + ' - ' + \
        repr(st.num_simplices()) + ' simplices - ' + \
        repr(st.num_vertices()) + ' vertices.' +\
        ' Filtration maximal value is ' + str(filtration_max) + '.'
    print(result_str)

    return st
def DTMFiltration(X, m, p, dimension_max, filtration_max = 8):
    
    DTM_values = DTM(X,X,m)
    st = WeightedRipsFiltration(X, DTM_values, p, dimension_max, filtration_max)
    return st
def AlphaWeightedRipsFiltration(X, F, p,dimension_max, filtration_max):
    N_tot = X.shape[0]     
    distances = euclidean_distances(X)          #compute the pairwise distances

    st_alpha = gudhi.AlphaComplex(points=X).create_simplex_tree()    
    st = gudhi.SimplexTree()                   #create an empty simplex tree
    
    for simplex in st_alpha.get_skeleton(2):            #add vertices with corresponding filtration value
        if len(simplex[0])==1:
            i = simplex[0][0]
            st.insert([i], filtration  = F[i])
        if len(simplex[0])==2:                     #add edges with corresponding filtration value
            i = simplex[0][0]
            j = simplex[0][1]
            value = Filtration_value(p, F[i], F[j], distances[i][j])
            st.insert([i,j], filtration  = value)

    st.expansion(dimension_max)                 #expand the complex

    result_str = 'Alpha Weighted Rips Complex is of dimension ' + repr(st.dimension()) + ' - ' + \
        repr(st.num_simplices()) + ' simplices - ' + \
        repr(st.num_vertices()) + ' vertices.' +\
        ' Filtration maximal value is ' + str(filtration_max) + '.'
    print(result_str)

    return st
def AlphaDTMFiltration(X, m, p, dimension_max, filtration_max = 8):
    DTM_values = DTM(X,X,m)
    st = AlphaWeightedRipsFiltration(X, DTM_values, p, dimension_max, filtration_max)
    return st

In [18]:
def fil(l,X,index):
 p ,m= 1,.1
 dimension_max = 2     
 st_DTM = DTMFiltration(l, m, p, dimension_max) 
 diagram_DTM = st_DTM.persistence()  
 st_alpha = gudhi.AlphaComplex(points=l).create_simplex_tree()  
 diagram_alpha = st_alpha.persistence()  
 b2,b3=0,0;
 for i in range(len(diagram_DTM)):
        if(diagram_DTM[i][0]==1):
            if(diagram_DTM[i][1][1]-diagram_DTM[i][1][0]>.1):
                b2+=1
        if(diagram_DTM[i][0]==0):
            if(diagram_DTM[i][1][1]-diagram_DTM[i][1][0]>.1):
                b3+=1
 X1[index].append(b2)
 X1[index].append(b3)
 barcodes(diagram_DTM)
 
 b2,b3=0,0;
 for i in range(len(diagram_alpha)):
        if(diagram_alpha[i][0]==1):
            if(diagram_alpha[i][1][1]-diagram_alpha[i][1][0]>.1):
                b2+=1
        if(diagram_alpha[i][0]==0):
            if(diagram_alpha[i][1][1]-diagram_alpha[i][1][0]>.1):
                b3+=1
 X1[index].append(b2)
 X1[index].append(b3)
 barcodes1(diagram_alpha)

In [ ]:
fil(X[71], 1.42,1)

In [ ]:
l5=l5[:100]
l6=l6[:100]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pylab as pl
Y=0
for i in range(len(l5)):
       x=[l5[i][0],l5[i][1]]
       y=[Y,Y]
       Y=Y+5
       if(i==len(l5)-1):
         plt.plot(x, y,c='b',label='H0')
       else:
         plt.plot(x, y,c='b')
for i in range(len(l6)):
       x=[l6[i][0],l6[i][1]]
       y=[Y,Y]
       Y=Y+5
       if(i==len(l6)-1):
         plt.plot(x, y,c='r',label='H1')
       else:
         plt.plot(x, y,c='r')
plt.title("Homology Barcodes\nAlpha filteration")
plt.xlabel('time')
pl.legend()
pl.show()

In [19]:
for i in range(len(X)):
         fil(X[i], 1.42,i)

Weighted Rips Complex is of dimension 2 - 294003 simplices - 1839 vertices. Filtration maximal value is 8.
Weighted Rips Complex is of dimension 2 - 330919 simplices - 1732 vertices. Filtration maximal value is 8.
Weighted Rips Complex is of dimension 2 - 248804 simplices - 1903 vertices. Filtration maximal value is 8.
Weighted Rips Complex is of dimension 2 - 282129 simplices - 1910 vertices. Filtration maximal value is 8.
Weighted Rips Complex is of dimension 2 - 797081 simplices - 1338 vertices. Filtration maximal value is 8.
Weighted Rips Complex is of dimension 2 - 943147 simplices - 1371 vertices. Filtration maximal value is 8.
Weighted Rips Complex is of dimension 2 - 279305 simplices - 1821 vertices. Filtration maximal value is 8.
Weighted Rips Complex is of dimension 2 - 352398 simplices - 1846 vertices. Filtration maximal value is 8.
Weighted Rips Complex is of dimension 2 - 1147062 simplices - 1348 vertices. Filtration maximal value is 8.
Weighted Rips Complex is of dimensio

Weighted Rips Complex is of dimension 2 - 1140530 simplices - 1253 vertices. Filtration maximal value is 8.
Weighted Rips Complex is of dimension 2 - 647097 simplices - 1426 vertices. Filtration maximal value is 8.
Weighted Rips Complex is of dimension 2 - 1278195 simplices - 1193 vertices. Filtration maximal value is 8.


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.decomposition import PCA as sklearnPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.datasets.samples_generator import make_blobs

In [21]:
X2=X1
for k in X2:
   for l in range(len(k)):
     k[l]=k[l]/sum(k)

In [ ]:
print(X1)

In [ ]:
import pylab as pl
for i in range(0,58,1):
    pca_2d=[max(X2[i]),min(X2[i])]
    pl.scatter(pca_2d[0],pca_2d[1],c='r', marker='o')
for i in range(59,89,1):
    pca_2d1=[max(X2[i]),min(X2[i])]
    pl.scatter(pca_2d1[0],pca_2d1[1],c='b', marker='*')
pl.title('Detection of Covid-19 Pnumonia')
axes = plt.gca()
axes.set_xlim([.9985,.9995])
plt.plot(pca_2d[0],pca_2d[1],c='r',label='Covid-19-PNumonia')
plt.plot(pca_2d1[0],pca_2d1[1],c='b',label='Normal-PNumonia')
pl.legend()
pl.show()

In [ ]:
import pylab as pl
for i in range(59,80,1):
    pca_2d=[max(X2[i]),min(X2[i])]
    pl.scatter(pca_2d[0],pca_2d[1],c='r', marker='o')
pl.title('Covid-19 Pnumonia')
plt.plot(pca_2d[0],pca_2d[1],c='r',label='IVP')
pl.legend()
pl.show()

In [ ]:
Y=[]
for i in range(59):
    Y.append(1)
for i in range(59,79,1):
    Y.append(0)


In [22]:
print(X1)

[[0.19457306908686794, 0.21386206602293353, 0.1490330743975607, 0.31605771878126676, 0.47187429142285736, 0.9993600479637785], [0.19590294351630866, 0.20912181949295175, 0.15417319093665027, 0.3201739617998992, 0.46412615188922096, 0.9993184860634629], [0.19359033906177428, 0.2134499110885082, 0.1417607348674602, 0.32471361720939024, 0.4732339204599972, 0.9993541848502456], [0.19033259423503326, 0.21173838309609444, 0.16605520205984114, 0.31825015726613265, 0.47533089170019915, 0.999365869651222], [0.19839978302142663, 0.20909465668507576, 0.1334643024682021, 0.3302447176580908, 0.460596655584249, 0.999090506594672], [0.19362126245847175, 0.21060633192750997, 0.14069793862437052, 0.3330462863398686, 0.46906672955557577, 0.9990763251041768], [0.19542306534176454, 0.2110726985436927, 0.13439996315374947, 0.3326306323551839, 0.466901765398488, 0.9993120154437882], [0.19139018371264052, 0.21464439009503186, 0.1584536071325791, 0.31568698450972593, 0.47652601190668054, 0.9993525201811251], 

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X1, Y, test_size=0.3, random_state=0)
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

NameError: name 'train_test_split' is not defined

In [ ]:
from mlxtend.plotting import plot_confusion_matrix
def confusionMat(clf,class_names,X,Y,y_pred,y_test,z):
    # Plot non-normalized confusion matrix
    titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]
    for title, normalize in titles_options:
            disp = plot_confusion_matrix(clf, X,Y,
                                 display_labels=class_names,
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title+"\n"+z)

    print(title)
    print(disp.confusion_matrix)
    print(classification_report(y_test, y_pred, target_names=class_names))
    sss=sensitivity_specificity_support(y_test, y_pred, average='macro')
    print("sensitivity:"+str(sss[0]))
    print("sensitivity:"+str(sss[1]))
    print("\n")
    plt.show()

In [ ]:
#from sklearn.metrics import plot_confusion_matrix
z="DT clasifier"+"Accuracy:"+str(metrics.accuracy_score(y_test, y_pred))
print(z)
class_names = ['Covid19-pnumonia','Normal-pnumonia']
confusionMat(clf,class_names,X,Y,y_pred,y_test,z)

In [ ]:
Y1=[1,2,2,3,3,3,3,3,3,1,1,1,1,2,2,2,3,3,3,3,3,3,3,3,1,1,3,1,1,1,2,1,2,3,1,2,2,2,3,3,3,3,2,2,2,2,2,2,3,1,2,3,3,3,3,3,1,1,1]

In [ ]:
X3=X1[:59]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X3, Y1, test_size=0.3, random_state=0)
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
print(y_test)
print(y_pred)

In [ ]:
from sklearn import svm
X_train, X_test, y_train, y_test = train_test_split(X1, Y, test_size=0.3, random_state=0)
clf = svm.SVC()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))